In [78]:
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd

In [79]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [80]:
x_train = tf.reshape(x_train/255, (-1, 784))
x_test = tf.reshape(x_test/255, (-1, 784))

x_train = tf.cast(x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)

In [81]:
def factor():
  W_ob =yield tfp.distributions.Normal(loc = tf.zeros([784,10]), scale = tf.ones([784,10]))
  B_ob =yield tfp.distributions.Normal(loc = tf.zeros([10]), scale =tf.ones([10]))
  Y_ob =yield tfp.distributions.Multinomial(total_count=1, probs = tf.nn.softmax(tf.matmul(x_train,W_ob)+B_ob))



In [82]:
model=tfp.distributions.JointDistributionCoroutineAutoBatched(factor)

In [83]:
w,b,y = model.sample()

In [84]:
new_W = tf.Variable(tf.ones([784,10]), dtype = tf.float32)
new_B = tf.Variable(tf.ones([10]), dtype = tf.float32)

In [85]:
from tensorflow.keras.utils import to_categorical
#One-Hot
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Y_train = tf.cast(y_train, tf.float32)

In [86]:
loss = lambda w,b:model.log_prob(w,b,y_train)

In [87]:
tfp.math.minimize(lambda : -loss(new_W,new_B), 500, tf.keras.optimizers.Adam(0.05))

<tf.Tensor: shape=(500,), dtype=float32, numpy=
array([149293.73 ,  85148.77 ,  66942.78 ,  57404.43 ,  46083.336,
        48217.5  ,  42764.797,  39932.773,  40427.133,  39781.9  ,
        37961.637,  36558.668,  36232.76 ,  35787.273,  34630.2  ,
        33835.54 ,  34065.945,  34354.16 ,  33738.41 ,  32636.355,
        31912.58 ,  31723.078,  31591.473,  31210.688,  30783.326,
        30588.117,  30533.33 ,  30279.771,  29771.385,  29343.146,
        29198.758,  29134.814,  28905.219,  28601.37 ,  28436.2  ,
        28361.078,  28182.414,  27906.127,  27696.121,  27584.44 ,
        27456.996,  27281.676,  27143.021,  27063.176,  26954.984,
        26782.594,  26633.629,  26552.975,  26473.371,  26363.617,
        26271.367,  26201.844,  26112.668,  26012.775,  25938.543,
        25881.508,  25813.34 ,  25736.898,  25676.11 ,  25625.906,
        25564.766,  25505.229,  25458.645,  25406.375,  25348.387,
        25303.133,  25264.586,  25218.977,  25173.268,  25134.32 ,
        25094.

In [88]:
_,_,pred = model.sample(value=(new_W,new_B))

In [89]:
import numpy as np
np.sum(tf.argmax(pred,axis=1)==tf.argmax(y_train,axis=1))/len(y_train)

0.8900166666666667

In [90]:
##testing
w,b,_ = model.sample(value=(new_W,new_B))

In [91]:
post_pred = tf.nn.softmax(tf.matmul(x_test, w)+b)

In [92]:
np.sum(tf.argmax(post_pred,axis=1)==tf.argmax(y_test,axis=1))/len(y_test)

0.9272

In [93]:
post_pred_onTrain = tf.nn.softmax(tf.matmul(x_train, w)+b)
np.sum(tf.argmax(post_pred_onTrain,axis=1)==tf.argmax(y_train,axis=1))/len(y_train)

0.9383833333333333